# Importing Libraries and data

In [1]:
import pandas as pd

In [2]:
data = 'https://gist.githubusercontent.com/cloudwalk-tests/76993838e65d7e0f988f40f1b1909c97/raw/295d9f7cb8fdf08f3cb3bdf1696ab245d5b5c1c9/transactional-sample.csv'

In [3]:
df = pd.read_csv(data)

# Dataset Overview

In [4]:
df.head()

,transaction_id,merchant_id,user_id,card_number,transaction_date,transaction_amount,device_id,has_cbk
0,21320398,29744,97051,434505******9116,2019-12-01T23:16:32.812632,374.56,285475.0,False
1,21320399,92895,2708,444456******4210,2019-12-01T22:45:37.873639,734.87,497105.0,True
2,21320400,47759,14777,425850******7024,2019-12-01T22:22:43.021495,760.36,NaN,False
3,21320401,68657,69758,464296******3991,2019-12-01T21:59:19.797129,2556.13,NaN,True
4,21320402,54075,64367,650487******6116,2019-12-01T21:30:53.347051,55.36,860232.0,False


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3199 entries, 0 to 3198
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   transaction_id      3199 non-null   int64  
 1   merchant_id         3199 non-null   int64  
 2   user_id             3199 non-null   int64  
 3   card_number         3199 non-null   object 
 4   transaction_date    3199 non-null   object 
 5   transaction_amount  3199 non-null   float64
 6   device_id           2369 non-null   float64
 7   has_cbk             3199 non-null   bool   
dtypes: bool(1), float64(2), int64(3), object(2)
memory usage: 178.2+ KB


In [57]:
print("Unique users:", df['user_id'].nunique())
print("Unique merchants:", df['merchant_id'].nunique())
print("Unique devices:", df['device_id'].nunique())
print("Chargeback rate:", df['has_cbk'].mean())

Unique users: 2704
Unique merchants: 1756
Unique devices: 1996
Chargeback rate: 0.12222569552985307


Checking if transaction_id column contains duplicates

In [12]:
df[df.duplicated(subset='transaction_id')]

,transaction_id,merchant_id,user_id,card_number,transaction_date,transaction_amount,device_id,has_cbk


Converting 'transaction_date' column to Datetime

In [13]:
df['transaction_date'] = pd.to_datetime(df['transaction_date'])

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3199 entries, 0 to 3198
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   transaction_id      3199 non-null   int64         
 1   merchant_id         3199 non-null   int64         
 2   user_id             3199 non-null   int64         
 3   card_number         3199 non-null   object        
 4   transaction_date    3199 non-null   datetime64[ns]
 5   transaction_amount  3199 non-null   float64       
 6   device_id           2369 non-null   float64       
 7   has_cbk             3199 non-null   bool          
dtypes: bool(1), datetime64[ns](1), float64(2), int64(3), object(1)
memory usage: 178.2+ KB


In [16]:
df.head()

,transaction_id,merchant_id,user_id,card_number,transaction_date,transaction_amount,device_id,has_cbk
0,21320398,29744,97051,434505******9116,2019-12-01 23:16:32.812632,374.56,285475.0,False
1,21320399,92895,2708,444456******4210,2019-12-01 22:45:37.873639,734.87,497105.0,True
2,21320400,47759,14777,425850******7024,2019-12-01 22:22:43.021495,760.36,NaN,False
3,21320401,68657,69758,464296******3991,2019-12-01 21:59:19.797129,2556.13,NaN,True
4,21320402,54075,64367,650487******6116,2019-12-01 21:30:53.347051,55.36,860232.0,False


# Suspicious Patterns

## CBK

CBK rate by merchant

In [64]:
merchant_cbk = df.groupby('merchant_id')['has_cbk'].mean().sort_values(ascending=False)
merchant_cbk.head(10)

merchant_id
15950    1.0
65241    1.0
67075    1.0
11911    1.0
5763     1.0
67764    1.0
68657    1.0
38568    1.0
70899    1.0
72723    1.0
Name: has_cbk, dtype: float64

In [77]:
merchant_cbk_stats = df.groupby('merchant_id').agg(
    cbk_rate=('has_cbk', 'mean'),
    txn_count=('has_cbk', 'count')
).sort_values(by=['cbk_rate', 'txn_count'], ascending=False)

merchant_cbk_stats.head(10)

,cbk_rate,txn_count
merchant_id,,
1308,1.0,15
44927,1.0,11
73271,1.0,10
38337,1.0,6
48126,1.0,6
75917,1.0,6
81795,1.0,6
8942,1.0,5
15326,1.0,5


CBK rate by user

In [65]:
user_cbk = df.groupby('user_id')['has_cbk'].mean().sort_values(ascending=False)
user_cbk.head(10)

user_id
39334    1.0
45031    1.0
11065    1.0
86219    1.0
50699    1.0
5541     1.0
47604    1.0
19820    1.0
21233    1.0
38734    1.0
Name: has_cbk, dtype: float64

In [75]:
user_cbk_stats = df.groupby('user_id').agg(
    cbk_rate=('has_cbk', 'mean'),
    txn_count=('has_cbk', 'count')
).sort_values(by=['cbk_rate', 'txn_count'], ascending=False)

In [76]:
user_cbk_stats.head(10)

,cbk_rate,txn_count
user_id,,
75710,1.0,10
7725,1.0,7
17929,1.0,6
21768,1.0,6
28218,1.0,5
71424,1.0,5
86411,1.0,5
99396,1.0,5
11065,1.0,4


CBK rate by device

In [68]:
device_cbk = df.groupby('device_id')['has_cbk'].mean().sort_values(ascending=False)
device_cbk.head(10)

device_id
6638.0      1.0
287404.0    1.0
306719.0    1.0
307489.0    1.0
486.0       1.0
69227.0     1.0
367746.0    1.0
122092.0    1.0
180611.0    1.0
308950.0    1.0
Name: has_cbk, dtype: float64

In [78]:
device_cbk_stats = df.groupby('device_id').agg(
    cbk_rate=('has_cbk', 'mean'),
    txn_count=('has_cbk', 'count')
).sort_values(by=['cbk_rate', 'txn_count'], ascending=False)

In [79]:
device_cbk_stats.head(10)

,cbk_rate,txn_count
device_id,,
357277.0,1.0,6
960729.0,1.0,6
174844.0,1.0,5
308950.0,1.0,5
597308.0,1.0,5
486.0,1.0,4
122092.0,1.0,4
288714.0,1.0,4
570971.0,1.0,4


In [80]:
df = df.sort_values(by=['card_number', 'transaction_date'])
df['prev_txn_time'] = df.groupby('card_number')['transaction_date'].shift()
df['time_diff'] = (df['transaction_date'] - df['prev_txn_time']).dt.total_seconds() / 60
suspicious_velocity = df[(df['time_diff'] >= 0) & (df['time_diff'] < 5)]  # <5 minutes

In [82]:
suspicious_velocity.head(10)

,transaction_id,merchant_id,user_id,card_number,transaction_date,transaction_amount,device_id,has_cbk,prev_txn_time,time_diff
756,21321154,48126,92259,406669******1721,2019-11-29 00:01:41.332782,1012.42,355583.0,True,2019-11-28 23:59:44.770666,1.942702
629,21321027,80155,88553,410863******7755,2019-11-29 15:35:24.506721,280.52,27250.0,False,2019-11-29 15:33:58.565067,1.432361
628,21321026,80155,88553,410863******7755,2019-11-29 15:36:16.283879,151.64,27250.0,False,2019-11-29 15:35:24.506721,0.862953
964,21321362,46045,10378,415944******1540,2019-11-28 15:41:40.107253,553.66,17372.0,True,2019-11-28 15:40:42.787730,0.955325
481,21320879,48126,55630,421960******7769,2019-11-29 21:07:27.923685,1074.53,611676.0,True,2019-11-29 21:04:05.545169,3.372975
480,21320878,48126,55630,421960******7769,2019-11-29 21:09:39.628804,1086.37,611676.0,True,2019-11-29 21:07:27.923685,2.195085
1173,21321571,11202,88625,427167******6030,2019-11-26 17:04:23.573237,80.52,917833.0,False,2019-11-26 17:02:57.542262,1.433850
130,21320528,93520,77959,432957******7262,2019-12-01 11:00:12.687222,2.75,589318.0,False,2019-12-01 10:59:39.973715,0.545225
129,21320527,93520,77959,432957******7262,2019-12-01 11:00:36.196460,2.00,589318.0,False,2019-12-01 11:00:12.687222,0.391821
1925,21322323,49387,98432,455184******9829,2019-11-22 15:18:41.555535,514.89,534341.0,False,2019-11-22 15:17:02.858448,1.644951


In [83]:
device_card_counts = df.groupby('device_id')['card_number'].nunique().sort_values(ascending=False)

In [84]:
device_card_counts.head(10)

device_id
563499.0    22
342890.0    19
101848.0    15
438940.0    10
547440.0    10
223682.0     7
542535.0     7
262327.0     6
960729.0     6
115003.0     5
Name: card_number, dtype: int64

In [85]:
avg_amount = df.groupby('has_cbk')['transaction_amount'].mean()

In [86]:
avg_amount

has_cbk
False     672.324380
True     1453.571918
Name: transaction_amount, dtype: float64

In [87]:
df['hour'] = df['transaction_date'].dt.hour
hourly_cbk = df.groupby('hour')['has_cbk'].mean()
print("\n--- Fraud rate by hour of day ---")
print(hourly_cbk)


--- Fraud rate by hour of day ---
hour
0     0.180451
1     0.126126
2     0.295082
3     0.133333
4     0.000000
5     0.000000
6     0.500000
8     0.000000
9     0.000000
10    0.000000
11    0.053763
12    0.081081
13    0.056410
14    0.064777
15    0.075314
16    0.107914
17    0.114391
18    0.082734
19    0.167939
20    0.136029
21    0.185022
22    0.177914
23    0.198864
Name: has_cbk, dtype: float64
